# ラボ 3.1: 外部評価パイプラインを使用して Langfuse トレースを評価する

#### 外部評価パイプラインは、次の場合に役立ちます：
- トレース評価のタイミングをより細かく制御できる。パイプラインを特定の時間に実行するようにスケジュールしたり、Webhook などのイベントベースのトリガーに応答させたりできる。
- Langfuse UI で可能な範囲を超えたカスタム評価が必要な場合、カスタム評価に柔軟性を持たせることができる。
- カスタム評価のバージョン管理
- 既存の評価フレームワークと事前定義されたメトリクスを使用してデータを評価する機能

このノートブックでは、以下の手順で外部評価パイプラインを実装する方法を学習します：
1. モデルをテストするための合成データセットを作成する
2. Langfuse クライアントを使用して、以前のモデル実行のトレースを収集およびフィルタリングする
3. これらのトレースをオフラインで、かつ段階的に評価する
4. 既存の Langfuse トレースにスコアを追加する

## 前提条件

> ℹ️ AWS が提供する一時アカウントを使用してインストラクター主導のワークショップに参加している場合は、**これらの前提条件の手順をスキップできます**

In [ ]:
# AWS ワークショップ環境を使用していない場合は、以下の行のコメントを外して依存関係をインストールしてください
# %pip install langfuse datasets ragas python-dotenv langchain-aws boto3 --upgrade

セルフホストまたはクラウドの Langfuse 環境に接続します。

In [ ]:
# すでに VS Code サーバーの .env で環境変数を定義している場合は、以下のセルはスキップしてください。
# langfuse 用の環境変数を定義してください。
# これらの値は Langfuse で API キーを作成する際に確認することができます。
# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "xxxx" # Langfuse プロジェクトのシークレットキー
# os.environ["LANGFUSE_PUBLIC_KEY"] = "xxxx" # Langfuse プロジェクトのパブリックキー
# os.environ["LANGFUSE_HOST"] = "xxx" # Langfuse ドメイン

## 初期化と認証チェック
以下のセルを実行して、共通ライブラリとクライアントを初期化してください。

In [ ]:
import sys
import os

import boto3
from langfuse.decorators import langfuse_context, observe

Amazon Bedrock クライアントを初期化し、アカウントで利用可能なモデルを確認します。

In [ ]:
# Amazon Bedrock の設定にアクセスするために利用
bedrock = boto3.client(service_name="bedrock", region_name="us-west-2")

# このリージョンで Nova モデルが利用可能か確認
models = bedrock.list_inference_profiles()
nova_found = False
for model in models["inferenceProfileSummaries"]:
    if (
        "Nova Pro" in model["inferenceProfileName"]
        or "Nova Lite" in model["inferenceProfileName"]
        or "Nova Micro" in model["inferenceProfileName"]
    ):
        print(
            f"Found Nova model: {model['inferenceProfileName']} - {model['inferenceProfileId']}"
        )
        nova_found = True
if not nova_found:
    raise ValueError(
        "No Nova models found in available models. Please ensure you have access to Nova models."
    )

Langfuse クライアントを初期化し、認証情報が有効であることを確認します。

In [ ]:
from langfuse import Langfuse

# langfuse クライアント
langfuse = Langfuse()
if langfuse.auth_check():
    print("Langfuse は正しく設定されています")
    print(f"Langfuse インスタンスへはこちらからアクセスできます: {os.environ['LANGFUSE_HOST']}")
else:
    print(
        "認証情報が見つからないか問題があります。.env ファイル内の Langfuse API キーとホストを確認してください。"
    )

### Amazon Bedrock Converse API の Langfuse ラッパー

In [ ]:
sys.path.append(os.path.abspath('..'))  # Add parent directory to path
from config import MODEL_CONFIG
from utils import converse

# 合成データの生成

このノートブックでは、LLM を活用して、e コマースページで製品に関してアドバイスする際に使用できる製品説明を生成するユースケースを検討します。最初のステップでは、製品のリストを生成し、それぞれの製品に対して Amazon Nova Lite に簡潔な製品説明を生成するよう指示します。

In [ ]:
# 50 製品を生成するプロンプト
messages = [
    {
        "role": "user",
        "content": "e コマースウェブサイトで販売されている 50 種類の異なる製品カテゴリーについて、\
        消費者にとって興味深い製品を 1 つずつ生成します。製品名は実際の販売されている製品を反映したものにする必要があります。 \
        50 の製品アイテムをカンマ区切りの値として生成します。製品名以外に追加の単語は生成しないでください。",
    },
]

# Nova Lite モデルの API 呼び出し
model_response = converse(messages=messages, **MODEL_CONFIG["nova_lite"])

# 生成されたテキストを表示
print("\n[Response Content Text]")
print(model_response)

In [ ]:
# モデルが生成した出力を確認
products = [item.strip() for item in model_response.split(",")]

for prd in products:
    print(prd)

次に、各製品について Amazon Nova Lite を使用して製品説明を生成し、```@observe()``` デコレータを使用して Langfuse にトレースをキャプチャします。

In [ ]:
# 各製品の製品説明を生成
prompt_template = "あなたはプロダクトマーケターであり、e コマースウェブサイトで \
製品を販売するために使用される詳細な製品説明を生成する必要があります。 \
説明のキャッチーなフレーズは、ソーシャルメディアキャンペーンにも使用されます。 \
製品説明から、お客様は製品が生活にどのように役立つかを理解できるだけでなく、 \
この会社を信頼できることも理解できるべきです。あなたの説明は楽しく魅力的です。 \
あなたの回答は最大 4 文にしてください。"


@observe(name="Batch Product Description Generation")
def main():
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="nova-batch-generation-session",
        tags=["lab3.1"],
    )

    for product in products:
        print(f"Input: {product} の説明文を生成してください。")
        messages = [
            {"role": "system", "content": prompt_template},
            {"role": "user", "content": f"{product} の説明文を生成してください。"},
        ]
        response = converse(
            messages, metadata={"product": product}, **MODEL_CONFIG["nova_lite"]
        )
        print(f"Output: {response} \n")


main()

langfuse_context.flush()

これで、langfuse UI のトレースセクションにこれらの製品説明が表示されるはずです。

![Traces collected from the LLM generations](./images/product_description_traces.png "Langfuse Traces")


このチュートリアルの目標は、モデルベースの評価パイプラインを構築する方法を示すことです。これらのパイプラインは、CI/CD 環境で実行されるか、異なるオーケストレーションされたコンテナサービスで実行されるでしょう。選択する環境に関係なく、常に 3 つの重要なステップが適用されます：

1. トレースを取得：アプリケーショントレースを評価環境に取得する
2. 評価を実行：任意の評価ロジックを適用する
3. 結果を保存：評価の計算に使用した Langfuse トレースに評価を戻して添付する

***
目標：この評価パイプラインは、過去 24 時間のすべてのトレースに対して実行されます
***

## 1. トレース の取得

`fetch_traces()` 関数には、タグ、タイムスタンプなどでトレースにフィルターをかける引数があります。また、ページネーション用のサンプル数を選択することもできます。

In [ ]:
from datetime import datetime, timedelta

now = datetime.now()
last_24_hours = now - timedelta(days=1)

traces_batch = langfuse.fetch_traces(
    page=1,
    limit=1,
    tags="lab3.1",
    session_id="nova-batch-generation-session",
    from_timestamp=last_24_hours,
    to_timestamp=datetime.now(),
).data

print(f"Trace ID: {traces_batch[0].id}")

In [ ]:
observations = langfuse.fetch_observations(trace_id=traces_batch[0].id, type="GENERATION").data
observations[0].output

## 2. LLM-as-a-judge を使用したカテゴリ評価

評価関数は入力としてトレースを受け取り、有効なスコアを出力します。
LLM アプリケーションの出力を分析する際、読みやすさ、有用性などの定性的に定義された特性や、完全性などの幻覚を軽減するための測定値を評価したい場合があります。

私たちは製品の説明を作成しており、顧客に響くものであることを確認するために、読みやすさを測定したいと考えています。LLM-as-a-judge の定義について詳しくは、[Amazon Bedrock 評価プロンプト](https://docs.aws.amazon.com/bedrock/latest/userguide/model-evaluation-type-judge-prompt.html) に定義されているジャッジベースの評価プロンプトをご覧ください。

In [ ]:
template_readability = """
あなたは、与えられた指示に従って LLM の応答を評価できる役立つエージェントです。

LLM によって生成された製品説明が与えられます。あなたの課題は、質問に対する LLM 応答の読みやすさを評価することです。つまり、一般的な読者が通常の読書速度で応答を理解しやすいかどうかを評価します。

以下の尺度に基づいて、応答の読みやすさを評価してください。
- unreadable: 応答はナンセンスであるか、通常の読者には理解できません。
- poor readability: 応答は理解できますが、理解を非常に困難にする読みにくい要因がたくさんあります。
- fair readability: 応答は理解できますが、読みにくい要因と読みやすい要因が混在しているため、平均的な読者はテキストを理解するためにある程度時間を費やす必要があります。
- good readability: 読みにくい要因はほとんどありません。ほとんどが明確で構造化された文章です。難しい単語にはコンテキストが明確に示されている標準的な語彙が使われています。トピック文と補足説明が明確に組織化されています。平均的な読者は一度通して読めば理解できます。
- excellent readability: 読みにくい要因はありません。文構造は一貫して明確、簡潔、かつ多様な表現が使われています。また、広く理解されている簡単な語彙が使われています。アイデアの間の移行がスムーズな論理的な構成になっています。平均的な読者は読み飛ばしても必要な点を理解できるかもしれません。

評価が必要な製品説明は次のとおりです: {prd_desc}

最初に応答の説明をし、次に最終的な回答を示してください。次の形式に従う必要があります。
Explanation: [説明], Answer: [回答],
ここで '[回答]' は以下のいずれかになります。
```
unreadable
poor readability
fair readability
good readability
excellent readability
```
"""

@observe(name="Readability Evaluation")
def generate_readability_score(trace_output):
    prd_desc_readability = template_readability.format(prd_desc=trace_output)
    # query = [f"Rate the readability of product description: {traces_batch[1].output}"]
    readability_score = converse(
            messages=[{"role": "user", "content": prd_desc_readability}], **MODEL_CONFIG["nova_pro"]
        )
    explanation, score = readability_score.split("\n\n")
    return explanation, score


print(f"ユーザーのクエリ: {observations[0].input[1]['content']}")
print(f"モデルの回答: {observations[0].output}")
explanation, score = generate_readability_score(observations[0].output)
print(f"読みやすさ: {score}, 説明: {explanation}")

## 3. トレースに評価を追加

読みやすさのスコアと説明を生成したので、Langfuse クライアントを使用して既存のトレースにスコアを追加できます。

In [ ]:
langfuse.score(
    trace_id=traces_batch[0].id,
    observation_id=observations[0].id,
    name="readability",
    value=score,
    comment=explanation,
)

# すべてを組み合わせる

1 つのトレースに対してこれを実行する方法を見てきました。過去 24 時間に収集されたすべてのトレースに対して実行する関数にまとめてみましょう。

In [ ]:
@observe(name="Batch Readability Evaluation")
def batch_evaluate():
    langfuse_context.update_current_trace(
        user_id="nova-user-1",
        session_id="nova-batch-evals-session",
        tags=["lab3.1"],
    )
    traces_batch = langfuse.fetch_traces(
        page=1,
        limit=1,
        tags="lab3.1",
        session_id="nova-batch-generation-session",
        from_timestamp=last_24_hours,
        to_timestamp=datetime.now(),
    ).data

    observations = langfuse.fetch_observations(trace_id=traces_batch[0].id, type="GENERATION").data

    for observation in observations[-5:]: # 時間の節約のために最新の 5 つのオブザベーションのみを評価
        print(f"Processing {observation.id}")
        if observation.output is None:
            print(
                f"Warning: \n Trace {observation.id} had no generated output, \
            it was skipped"
            )
            continue
        explanation, score = generate_readability_score(observation.output)
        langfuse.score(
            trace_id=traces_batch[0].id,
            observation_id=observation.id,
            name="readability",
            value=score,
            comment=explanation,
        )

In [ ]:
batch_evaluate()

langfuse_context.flush()

#### パイプラインが正常に実行された場合、トレースにスコアが追加されているはずです

![読みやすさのスコアが追加された Langfuse トレース](./images/scored_trace.png "Langfuse でのスコア付きトレース")

### おめでとうございます
ラボ 3.1 を無事終了しました。

AWS イベントに参加している場合は、次のラボに進む前に、ワークショップスタジオに戻って追加の指示を受けてください。次のラボでは、GenAI ガードレールについて探求します。